# Kaggle Competition : Microsoft Malware Data
> This is a Sungryong Hong's Notebook.  

> I have a stand-alone Spark(2.3.2)/Hadoop(2.8.3) cluster, which has 48 logical cores with 150GB memory. 

> I have put the data files to my hdfs. Check the contents as `hfs -cat /data/spark/msmalware/test.csv | head`.  

>`hfs` is an alias for `hdfs dfs`. 


## 1. Import Basic Packages

In [1]:
# Basic Libraries 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import gc

pd.set_option('display.max_rows', 500)

# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

#### Spark-Shell Sesssion 

In [2]:
# Basic PySpark Libraries

# Old Style : SparkContext 
#from pyspark import SparkContext   
#from pyspark.sql import SQLContext


# New Style : Spark Session  
#Shell-Mode: Spark Session Name is `spark`

sc = spark.sparkContext
sqlsc = SQLContext(sc)
sc.setCheckpointDir("hdfs://master:54310/tmp/spark/checkpoints")

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

In [3]:
# Enable Arrow for boosting up python performances 
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#### Spark-Submit Session

> An example of Spark Session for non-shell (submit) mode; 

>`spark = SparkSession.builder.appName("largeScaleGstat")\
    .config("spark.driver.maxResultSize","8g")\
    .config("spark.sql.execution.arrow.enabled","true")\
    .config("spark.executor.memoryOverhead","42GB")\
    .getOrCreate()`


## 2. Read the *test.csv* 

### 2.1 Read the file to a spark dataframe

In [4]:
rawdf_schema = T.StructType([\
                             T.StructField('MachineIdentifier', T.StringType(), True),\
                             T.StructField('ProductName', T.StringType(), True),\
                             T.StructField('EngineVersion', T.StringType(), True),\
                             T.StructField('AppVersion', T.StringType(), True),\
                             T.StructField('AvSigVersion', T.StringType(), True),\
                             T.StructField('IsBeta', T.IntegerType(), True),\
                             T.StructField('RtpStateBitfield', T.StringType(), True),\
                             T.StructField('IsSxsPassiveMode', T.IntegerType(), True),\
                             T.StructField('DefaultBrowsersIdentifier', T.StringType(), True),\
                             T.StructField('AVProductStatesIdentifier', T.StringType(), True),\
                             T.StructField('AVProductsInstalled', T.StringType(), True),\
                             T.StructField('AVProductsEnabled', T.StringType(), True),\
                             T.StructField('HasTpm', T.IntegerType(), True),\
                             T.StructField('CountryIdentifier', T.IntegerType(), True),\
                             T.StructField('CityIdentifier', T.StringType(), True),\
                             T.StructField('OrganizationIdentifier', T.StringType(), True),\
                             T.StructField('GeoNameIdentifier', T.StringType(), True),\
                             T.StructField('LocaleEnglishNameIdentifier', T.IntegerType(), True),\
                             T.StructField('Platform', T.StringType(), True),\
                             T.StructField('Processor', T.StringType(), True),\
                             T.StructField('OsVer', T.StringType(), True),\
                             T.StructField('OsBuild', T.IntegerType(), True),\
                             T.StructField('OsSuite', T.IntegerType(), True),\
                             T.StructField('OsPlatformSubRelease', T.StringType(), True),\
                             T.StructField('OsBuildLab', T.StringType(), True),\
                             T.StructField('SkuEdition', T.StringType(), True),\
                             T.StructField('IsProtected', T.StringType(), True),\
                             T.StructField('AutoSampleOptIn', T.IntegerType(), True),\
                             T.StructField('PuaMode', T.StringType(), True),\
                             T.StructField('SMode', T.StringType(), True),\
                             T.StructField('IeVerIdentifier', T.StringType(), True),\
                             T.StructField('SmartScreen', T.StringType(), True),\
                             T.StructField('Firewall', T.StringType(), True),\
                             T.StructField('UacLuaenable', T.StringType(), True),\
                             T.StructField('Census_MDC2FormFactor', T.StringType(), True),\
                             T.StructField('Census_DeviceFamily', T.StringType(), True),\
                             T.StructField('Census_OEMNameIdentifier', T.StringType(), True),\
                             T.StructField('Census_OEMModelIdentifier', T.StringType(), True),\
                             T.StructField('Census_ProcessorCoreCount', T.FloatType(), True),\
                             T.StructField('Census_ProcessorManufacturerIdentifier', T.StringType(), True),\
                             T.StructField('Census_ProcessorModelIdentifier', T.StringType(), True),\
                             T.StructField('Census_ProcessorClass', T.StringType(), True),\
                             T.StructField('Census_PrimaryDiskTotalCapacity', T.FloatType(), True),\
                             T.StructField('Census_PrimaryDiskTypeName', T.StringType(), True),\
                             T.StructField('Census_SystemVolumeTotalCapacity', T.FloatType(), True),\
                             T.StructField('Census_HasOpticalDiskDrive', T.IntegerType(), True),\
                             T.StructField('Census_TotalPhysicalRAM', T.FloatType(), True),\
                             T.StructField('Census_ChassisTypeName', T.StringType(), True),\
                             T.StructField('Census_InternalPrimaryDiagonalDisplaySizeInInches', T.FloatType(), True),\
                             T.StructField('Census_InternalPrimaryDisplayResolutionHorizontal', T.FloatType(), True),\
                             T.StructField('Census_InternalPrimaryDisplayResolutionVertical', T.FloatType(), True),\
                             T.StructField('Census_PowerPlatformRoleName', T.StringType(), True),\
                             T.StructField('Census_InternalBatteryType', T.StringType(), True),\
                             T.StructField('Census_InternalBatteryNumberOfCharges', T.FloatType(), True),\
                             T.StructField('Census_OSVersion', T.StringType(), True),\
                             T.StructField('Census_OSArchitecture', T.StringType(), True),\
                             T.StructField('Census_OSBranch', T.StringType(), True),\
                             T.StructField('Census_OSBuildNumber', T.IntegerType(), True),\
                             T.StructField('Census_OSBuildRevision', T.IntegerType(), True),\
                             T.StructField('Census_OSEdition', T.StringType(), True),\
                             T.StructField('Census_OSSkuName', T.StringType(), True),\
                             T.StructField('Census_OSInstallTypeName', T.StringType(), True),\
                             T.StructField('Census_OSInstallLanguageIdentifier', T.StringType(), True),\
                             T.StructField('Census_OSUILocaleIdentifier', T.IntegerType(), True),\
                             T.StructField('Census_OSWUAutoUpdateOptionsName', T.StringType(), True),\
                             T.StructField('Census_IsPortableOperatingSystem', T.IntegerType(), True),\
                             T.StructField('Census_GenuineStateName', T.StringType(), True),\
                             T.StructField('Census_ActivationChannel', T.StringType(), True),\
                             T.StructField('Census_IsFlightingInternal', T.StringType(), True),\
                             T.StructField('Census_IsFlightsDisabled', T.StringType(), True),\
                             T.StructField('Census_FlightRing', T.StringType(), True),\
                             T.StructField('Census_ThresholdOptIn', T.StringType(), True),\
                             T.StructField('Census_FirmwareManufacturerIdentifier', T.StringType(), True),\
                             T.StructField('Census_FirmwareVersionIdentifier', T.StringType(), True),\
                             T.StructField('Census_IsSecureBootEnabled', T.IntegerType(), True),\
                             T.StructField('Census_IsWIMBootEnabled', T.StringType(), True),\
                             T.StructField('Census_IsVirtualDevice', T.StringType(), True),\
                             T.StructField('Census_IsTouchEnabled', T.IntegerType(), True),\
                             T.StructField('Census_IsPenCapable', T.IntegerType(), True),\
                             T.StructField('Census_IsAlwaysOnAlwaysConnectedCapable', T.StringType(), True),\
                             T.StructField('Wdft_IsGamer', T.StringType(), True),\
                             T.StructField('Wdft_RegionIdentifier', T.StringType(), True),\
                             T.StructField('HasDetections', T.IntegerType(), True)
                            ])


In [5]:
rawdf = sqlsc.read.csv("hdfs://master:54310/data/spark/msmalware/train.csv",\
                       header=True, schema = rawdf_schema)

### 2.2 Browse the raw data

In [6]:
rawdf.select('MachineIdentifier','ProductName','EngineVersion','AppVersion','AvSigVersion','IsBeta')\
     .show(10,truncate=True)

+--------------------+------------+-------------+---------------+------------+------+
|   MachineIdentifier| ProductName|EngineVersion|     AppVersion|AvSigVersion|IsBeta|
+--------------------+------------+-------------+---------------+------------+------+
|0000028988387b115...|win8defender|  1.1.15100.1|4.18.1807.18075|1.273.1735.0|     0|
|000007535c3f730ef...|win8defender|  1.1.14600.4|   4.13.17134.1|  1.263.48.0|     0|
|000007905a28d863f...|win8defender|  1.1.15100.1|4.18.1807.18075|1.273.1341.0|     0|
|00000b11598a75ea8...|win8defender|  1.1.15100.1|4.18.1807.18075|1.273.1527.0|     0|
|000014a5f00daa18e...|win8defender|  1.1.15100.1|4.18.1807.18075|1.273.1379.0|     0|
|000016191b897145d...|win8defender|  1.1.15100.1|4.18.1807.18075|1.273.1094.0|     0|
|0000161e8abf8d8b8...|win8defender|  1.1.15100.1|4.18.1807.18075| 1.273.845.0|     0|
|000019515bc8f9585...|win8defender|  1.1.15100.1|4.18.1807.18075|1.273.1393.0|     0|
|00001a027a0ab970c...|win8defender|  1.1.15200.1|4.18.

In [7]:
spark.conf.set('spark.debug.maxToStringFields',50)

> The default StringField size is 25. I got some warnings about this short string-field size. 

In [8]:
len(rawdf.columns)

83

In [9]:
rawdf.head(3)

[Row(MachineIdentifier=u'0000028988387b115f69f31a3bf04f09', ProductName=u'win8defender', EngineVersion=u'1.1.15100.1', AppVersion=u'4.18.1807.18075', AvSigVersion=u'1.273.1735.0', IsBeta=0, RtpStateBitfield=u'7', IsSxsPassiveMode=0, DefaultBrowsersIdentifier=None, AVProductStatesIdentifier=u'53447', AVProductsInstalled=u'1', AVProductsEnabled=u'1', HasTpm=1, CountryIdentifier=29, CityIdentifier=u'128035', OrganizationIdentifier=u'18', GeoNameIdentifier=u'35', LocaleEnglishNameIdentifier=171, Platform=u'windows10', Processor=u'x64', OsVer=u'10.0.0.0', OsBuild=17134, OsSuite=256, OsPlatformSubRelease=u'rs4', OsBuildLab=u'17134.1.amd64fre.rs4_release.180410-1804', SkuEdition=u'Pro', IsProtected=u'1', AutoSampleOptIn=0, PuaMode=None, SMode=u'0', IeVerIdentifier=u'137', SmartScreen=None, Firewall=u'1', UacLuaenable=u'1', Census_MDC2FormFactor=u'Desktop', Census_DeviceFamily=u'Windows.Desktop', Census_OEMNameIdentifier=u'2668', Census_OEMModelIdentifier=u'9124', Census_ProcessorCoreCount=4.0

> 83 columns ... kind of too many

#### Checking `Null` counts 

In [10]:
nullstatdf = rawdf.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in rawdf.columns])\
     .toPandas().transpose()
nullstatdf.columns = ['null_counts']
nullstatdf.sort_values(by='null_counts',ascending=False)

,null_counts
PuaMode,8919174
Census_ProcessorClass,8884852
DefaultBrowsersIdentifier,8488045
Census_IsFlightingInternal,7408759
Census_InternalBatteryType,6338414
Census_ThresholdOptIn,5667325
Census_IsWIMBootEnabled,5659703
SmartScreen,3177011
OrganizationIdentifier,2751518
SMode,537759


#### Checking `HasDetections` 

In [11]:
rawdf.groupby('HasDetections').count().show()

+-------------+-------+
|HasDetections|  count|
+-------------+-------+
|            1|4458892|
|            0|4462591|
+-------------+-------+



> Ok ... the data set is well-cleaned, showing a 50:50 categorical distribution.  

#### Checking Duplicates 

In [12]:
rawdf.count()

8921483

In [13]:
rawdf.dropDuplicates().count()

8921483

In [14]:
rawdf.agg(F.count('MachineIdentifier').alias('count'),F.countDistinct('MachineIdentifier').alias('distinct')).show()

+-------+--------+
|  count|distinct|
+-------+--------+
|8921483| 8921483|
+-------+--------+



> Again, this is a well-cleaned data set; no annoying duplicates or glitches. 

#### Explore some `crosstabs`  

In [15]:
rawdf.cube('EngineVersion','AppVersion','AvSigVersion','IsBeta').count().orderBy('count',ascending=False).show(10)

+-------------+---------------+------------+------+-------+
|EngineVersion|     AppVersion|AvSigVersion|IsBeta|  count|
+-------------+---------------+------------+------+-------+
|         null|           null|        null|  null|8921483|
|         null|           null|        null|     0|8921416|
|         null|4.18.1807.18075|        null|  null|5139224|
|         null|4.18.1807.18075|        null|     0|5139218|
|  1.1.15200.1|           null|        null|  null|3845067|
|  1.1.15200.1|           null|        null|     0|3845043|
|  1.1.15100.1|           null|        null|  null|3675915|
|  1.1.15100.1|           null|        null|     0|3675891|
|  1.1.15200.1|4.18.1807.18075|        null|  null|2915372|
|  1.1.15200.1|4.18.1807.18075|        null|     0|2915368|
+-------------+---------------+------------+------+-------+
only showing top 10 rows



In [16]:
rawdf.filter(rawdf.HasDetections ==1)\
     .cube('EngineVersion','AppVersion','AvSigVersion','IsBeta')\
     .count().orderBy('count',ascending=False).show(10)

+-------------+---------------+------------+------+-------+
|EngineVersion|     AppVersion|AvSigVersion|IsBeta|  count|
+-------------+---------------+------------+------+-------+
|         null|           null|        null|  null|4458892|
|         null|           null|        null|     0|4458859|
|         null|4.18.1807.18075|        null|  null|2725768|
|         null|4.18.1807.18075|        null|     0|2725766|
|  1.1.15100.1|           null|        null|  null|2031651|
|  1.1.15100.1|           null|        null|     0|2031637|
|  1.1.15200.1|           null|        null|  null|1890743|
|  1.1.15200.1|           null|        null|     0|1890733|
|  1.1.15200.1|4.18.1807.18075|        null|  null|1440624|
|  1.1.15200.1|4.18.1807.18075|        null|     0|1440623|
+-------------+---------------+------------+------+-------+
only showing top 10 rows



#### Explore categorical cardinalities and frequencies

In [17]:
rawdf.dtypes

[('MachineIdentifier', 'string'),
 ('ProductName', 'string'),
 ('EngineVersion', 'string'),
 ('AppVersion', 'string'),
 ('AvSigVersion', 'string'),
 ('IsBeta', 'int'),
 ('RtpStateBitfield', 'string'),
 ('IsSxsPassiveMode', 'int'),
 ('DefaultBrowsersIdentifier', 'string'),
 ('AVProductStatesIdentifier', 'string'),
 ('AVProductsInstalled', 'string'),
 ('AVProductsEnabled', 'string'),
 ('HasTpm', 'int'),
 ('CountryIdentifier', 'int'),
 ('CityIdentifier', 'string'),
 ('OrganizationIdentifier', 'string'),
 ('GeoNameIdentifier', 'string'),
 ('LocaleEnglishNameIdentifier', 'int'),
 ('Platform', 'string'),
 ('Processor', 'string'),
 ('OsVer', 'string'),
 ('OsBuild', 'int'),
 ('OsSuite', 'int'),
 ('OsPlatformSubRelease', 'string'),
 ('OsBuildLab', 'string'),
 ('SkuEdition', 'string'),
 ('IsProtected', 'string'),
 ('AutoSampleOptIn', 'int'),
 ('PuaMode', 'string'),
 ('SMode', 'string'),
 ('IeVerIdentifier', 'string'),
 ('SmartScreen', 'string'),
 ('Firewall', 'string'),
 ('UacLuaenable', 'string

In [18]:
%%time
listCategorical = []
listNumerical = []
for eachcol in rawdf.dtypes[1:]: # excluding Machine-Identifier from features
    if eachcol[1] == 'string':
        listCategorical.append([eachcol[0],rawdf.select(eachcol[0]).distinct().count()])
        print listCategorical[-1]
    else:
        listNumerical.append(eachcol[0])

['ProductName', 6]
['EngineVersion', 70]
['AppVersion', 110]
['AvSigVersion', 8531]
['RtpStateBitfield', 8]
['DefaultBrowsersIdentifier', 2018]
['AVProductStatesIdentifier', 28971]
['AVProductsInstalled', 9]
['AVProductsEnabled', 7]
['CityIdentifier', 107367]
['OrganizationIdentifier', 50]
['GeoNameIdentifier', 293]
['Platform', 4]
['Processor', 3]
['OsVer', 58]
['OsPlatformSubRelease', 9]
['OsBuildLab', 664]
['SkuEdition', 8]
['IsProtected', 3]
['PuaMode', 3]
['SMode', 3]
['IeVerIdentifier', 304]
['SmartScreen', 22]
['Firewall', 3]
['UacLuaenable', 12]
['Census_MDC2FormFactor', 13]
['Census_DeviceFamily', 3]
['Census_OEMNameIdentifier', 3833]
['Census_OEMModelIdentifier', 175366]
['Census_ProcessorManufacturerIdentifier', 8]
['Census_ProcessorModelIdentifier', 3429]
['Census_ProcessorClass', 4]
['Census_PrimaryDiskTypeName', 5]
['Census_ChassisTypeName', 53]
['Census_PowerPlatformRoleName', 11]
['Census_InternalBatteryType', 80]
['Census_OSVersion', 469]
['Census_OSArchitecture', 3]
[

In [19]:
print [eachcat[0] for eachcat in listCategorical]

['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'RtpStateBitfield', 'DefaultBrowsersIdentifier', 'AVProductStatesIdentifier', 'AVProductsInstalled', 'AVProductsEnabled', 'CityIdentifier', 'OrganizationIdentifier', 'GeoNameIdentifier', 'Platform', 'Processor', 'OsVer', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'IsProtected', 'PuaMode', 'SMode', 'IeVerIdentifier', 'SmartScreen', 'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorModelIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSVersion', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_A

In [20]:
len(listCategorical)

57

In [21]:
print listNumerical

['IsBeta', 'IsSxsPassiveMode', 'HasTpm', 'CountryIdentifier', 'LocaleEnglishNameIdentifier', 'OsBuild', 'OsSuite', 'AutoSampleOptIn', 'Census_ProcessorCoreCount', 'Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity', 'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM', 'Census_InternalPrimaryDiagonalDisplaySizeInInches', 'Census_InternalPrimaryDisplayResolutionHorizontal', 'Census_InternalPrimaryDisplayResolutionVertical', 'Census_InternalBatteryNumberOfCharges', 'Census_OSBuildNumber', 'Census_OSBuildRevision', 'Census_OSUILocaleIdentifier', 'Census_IsPortableOperatingSystem', 'Census_IsSecureBootEnabled', 'Census_IsTouchEnabled', 'Census_IsPenCapable', 'HasDetections']


In [22]:
len(listNumerical)

25

### 2.3 Applying `StringIndexer` to `listCategorical`

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

In [24]:
indexersCategorical = \
[StringIndexer(inputCol=eachcat[0], outputCol="{0}_indexed".format(eachcat[0]), handleInvalid='keep') \
 for eachcat in listCategorical]

In [25]:
print [eachindexer.getOutputCol() for eachindexer in indexersCategorical]

['ProductName_indexed', 'EngineVersion_indexed', 'AppVersion_indexed', 'AvSigVersion_indexed', 'RtpStateBitfield_indexed', 'DefaultBrowsersIdentifier_indexed', 'AVProductStatesIdentifier_indexed', 'AVProductsInstalled_indexed', 'AVProductsEnabled_indexed', 'CityIdentifier_indexed', 'OrganizationIdentifier_indexed', 'GeoNameIdentifier_indexed', 'Platform_indexed', 'Processor_indexed', 'OsVer_indexed', 'OsPlatformSubRelease_indexed', 'OsBuildLab_indexed', 'SkuEdition_indexed', 'IsProtected_indexed', 'PuaMode_indexed', 'SMode_indexed', 'IeVerIdentifier_indexed', 'SmartScreen_indexed', 'Firewall_indexed', 'UacLuaenable_indexed', 'Census_MDC2FormFactor_indexed', 'Census_DeviceFamily_indexed', 'Census_OEMNameIdentifier_indexed', 'Census_OEMModelIdentifier_indexed', 'Census_ProcessorManufacturerIdentifier_indexed', 'Census_ProcessorModelIdentifier_indexed', 'Census_ProcessorClass_indexed', 'Census_PrimaryDiskTypeName_indexed', 'Census_ChassisTypeName_indexed', 'Census_PowerPlatformRoleName_in

#### Sanity Check for `StringIndexer`

In [26]:
for idx in range(len(listCategorical)):
    indexersCategorical[idx].fit(rawdf).transform(rawdf)\
        .select('MachineIdentifier',listCategorical[idx][0],indexersCategorical[idx].getOutputCol()).show(5)

+--------------------+------------+-------------------+
|   MachineIdentifier| ProductName|ProductName_indexed|
+--------------------+------------+-------------------+
|0000028988387b115...|win8defender|                0.0|
|000007535c3f730ef...|win8defender|                0.0|
|000007905a28d863f...|win8defender|                0.0|
|00000b11598a75ea8...|win8defender|                0.0|
|000014a5f00daa18e...|win8defender|                0.0|
+--------------------+------------+-------------------+
only showing top 5 rows

+--------------------+-------------+---------------------+
|   MachineIdentifier|EngineVersion|EngineVersion_indexed|
+--------------------+-------------+---------------------+
|0000028988387b115...|  1.1.15100.1|                  1.0|
|000007535c3f730ef...|  1.1.14600.4|                  4.0|
|000007905a28d863f...|  1.1.15100.1|                  1.0|
|00000b11598a75ea8...|  1.1.15100.1|                  1.0|
|000014a5f00daa18e...|  1.1.15100.1|                  1.0|

+--------------------+--------+-------------+
|   MachineIdentifier|   OsVer|OsVer_indexed|
+--------------------+--------+-------------+
|0000028988387b115...|10.0.0.0|          0.0|
|000007535c3f730ef...|10.0.0.0|          0.0|
|000007905a28d863f...|10.0.0.0|          0.0|
|00000b11598a75ea8...|10.0.0.0|          0.0|
|000014a5f00daa18e...|10.0.0.0|          0.0|
+--------------------+--------+-------------+
only showing top 5 rows

+--------------------+--------------------+----------------------------+
|   MachineIdentifier|OsPlatformSubRelease|OsPlatformSubRelease_indexed|
+--------------------+--------------------+----------------------------+
|0000028988387b115...|                 rs4|                         0.0|
|000007535c3f730ef...|                 rs4|                         0.0|
|000007905a28d863f...|                 rs4|                         0.0|
|00000b11598a75ea8...|                 rs4|                         0.0|
|000014a5f00daa18e...|                 rs4|       

+--------------------+--------------------------------------+----------------------------------------------+
|   MachineIdentifier|Census_ProcessorManufacturerIdentifier|Census_ProcessorManufacturerIdentifier_indexed|
+--------------------+--------------------------------------+----------------------------------------------+
|0000028988387b115...|                                     5|                                           0.0|
|000007535c3f730ef...|                                     5|                                           0.0|
|000007905a28d863f...|                                     5|                                           0.0|
|00000b11598a75ea8...|                                     5|                                           0.0|
|000014a5f00daa18e...|                                     5|                                           0.0|
+--------------------+--------------------------------------+----------------------------------------------+
only showing top 5 

+--------------------+----------------+------------------------+
|   MachineIdentifier|Census_OSSkuName|Census_OSSkuName_indexed|
+--------------------+----------------+------------------------+
|0000028988387b115...|    PROFESSIONAL|                     1.0|
|000007535c3f730ef...|    PROFESSIONAL|                     1.0|
|000007905a28d863f...|            CORE|                     0.0|
|00000b11598a75ea8...|    PROFESSIONAL|                     1.0|
|000014a5f00daa18e...|            CORE|                     0.0|
+--------------------+----------------+------------------------+
only showing top 5 rows

+--------------------+------------------------+--------------------------------+
|   MachineIdentifier|Census_OSInstallTypeName|Census_OSInstallTypeName_indexed|
+--------------------+------------------------+--------------------------------+
|0000028988387b115...|              UUPUpgrade|                             0.0|
|000007535c3f730ef...|                IBSClean|                   

+--------------------+--------------------------------+----------------------------------------+
|   MachineIdentifier|Census_FirmwareVersionIdentifier|Census_FirmwareVersionIdentifier_indexed|
+--------------------+--------------------------------+----------------------------------------+
|0000028988387b115...|                           36144|                                  2516.0|
|000007535c3f730ef...|                           57858|                                  1766.0|
|000007905a28d863f...|                           52682|                                   189.0|
|00000b11598a75ea8...|                           20050|                                    32.0|
|000014a5f00daa18e...|                           19844|                                   123.0|
+--------------------+--------------------------------+----------------------------------------+
only showing top 5 rows

+--------------------+-----------------------+-------------------------------+
|   MachineIdentifier|C

#### Indexing all categorical features

In [27]:
%%time
pipeline = Pipeline(stages=indexersCategorical)
numdf = pipeline.fit(rawdf).transform(rawdf)

CPU times: user 444 ms, sys: 105 ms, total: 548 ms
Wall time: 10min 10s


In [28]:
numdf.cache()

DataFrame[MachineIdentifier: string, ProductName: string, EngineVersion: string, AppVersion: string, AvSigVersion: string, IsBeta: int, RtpStateBitfield: string, IsSxsPassiveMode: int, DefaultBrowsersIdentifier: string, AVProductStatesIdentifier: string, AVProductsInstalled: string, AVProductsEnabled: string, HasTpm: int, CountryIdentifier: int, CityIdentifier: string, OrganizationIdentifier: string, GeoNameIdentifier: string, LocaleEnglishNameIdentifier: int, Platform: string, Processor: string, OsVer: string, OsBuild: int, OsSuite: int, OsPlatformSubRelease: string, OsBuildLab: string, SkuEdition: string, IsProtected: string, AutoSampleOptIn: int, PuaMode: string, SMode: string, IeVerIdentifier: string, SmartScreen: string, Firewall: string, UacLuaenable: string, Census_MDC2FormFactor: string, Census_DeviceFamily: string, Census_OEMNameIdentifier: string, Census_OEMModelIdentifier: string, Census_ProcessorCoreCount: float, Census_ProcessorManufacturerIdentifier: string, Census_Proces

### 2.4 Applying `Imputer` to `listNumerical`

In [29]:
nullnumstatdf = rawdf.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in listNumerical])\
     .toPandas().transpose()
nullnumstatdf.columns = ['null_counts']
nullnumstatdf.sort_values(by='null_counts',ascending=False)

,null_counts
Census_InternalBatteryNumberOfCharges,268755
Census_TotalPhysicalRAM,80533
Census_PrimaryDiskTotalCapacity,53016
Census_SystemVolumeTotalCapacity,53002
Census_InternalPrimaryDiagonalDisplaySizeInInches,47134
Census_InternalPrimaryDisplayResolutionVertical,46986
Census_InternalPrimaryDisplayResolutionHorizontal,46986
Census_ProcessorCoreCount,41306
Census_IsPenCapable,0
Census_IsTouchEnabled,0


In [30]:
from pyspark.ml.feature import Imputer

In [31]:
listImputable = [eachcol[0] for eachcol in numdf.dtypes if eachcol[1] == 'float']

In [32]:
listImputable

['Census_ProcessorCoreCount',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_TotalPhysicalRAM',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_InternalBatteryNumberOfCharges']

In [33]:
listImputable_imputed = [numcol + "_imputed" for numcol in listImputable]

In [34]:
imputer = Imputer(inputCols = listImputable, outputCols = listImputable_imputed)

In [35]:
%%time
numdf = imputer.fit(numdf).transform(numdf)

CPU times: user 30 ms, sys: 12.3 ms, total: 42.3 ms
Wall time: 2min 2s


### 2.5 Checking the indexed/imputed dataframe 

In [36]:
numdf.cache()

DataFrame[MachineIdentifier: string, ProductName: string, EngineVersion: string, AppVersion: string, AvSigVersion: string, IsBeta: int, RtpStateBitfield: string, IsSxsPassiveMode: int, DefaultBrowsersIdentifier: string, AVProductStatesIdentifier: string, AVProductsInstalled: string, AVProductsEnabled: string, HasTpm: int, CountryIdentifier: int, CityIdentifier: string, OrganizationIdentifier: string, GeoNameIdentifier: string, LocaleEnglishNameIdentifier: int, Platform: string, Processor: string, OsVer: string, OsBuild: int, OsSuite: int, OsPlatformSubRelease: string, OsBuildLab: string, SkuEdition: string, IsProtected: string, AutoSampleOptIn: int, PuaMode: string, SMode: string, IeVerIdentifier: string, SmartScreen: string, Firewall: string, UacLuaenable: string, Census_MDC2FormFactor: string, Census_DeviceFamily: string, Census_OEMNameIdentifier: string, Census_OEMModelIdentifier: string, Census_ProcessorCoreCount: float, Census_ProcessorManufacturerIdentifier: string, Census_Proces

In [37]:
len(numdf.columns)

148

In [38]:
rawdf.select('EngineVersion').groupBy('EngineVersion').count().sort(F.desc("count")).show(10)

+-------------+-------+
|EngineVersion|  count|
+-------------+-------+
|  1.1.15200.1|3845067|
|  1.1.15100.1|3675915|
|  1.1.15000.2| 265218|
|  1.1.14901.4| 212408|
|  1.1.14600.4| 160585|
|  1.1.14800.3| 136476|
|  1.1.15300.6| 120295|
|  1.1.14104.0|  93926|
|  1.1.13504.0|  70645|
|  1.1.15300.5|  68716|
+-------------+-------+
only showing top 10 rows



In [39]:
numdf.select('EngineVersion_indexed').groupBy('EngineVersion_indexed').count().sort(F.desc("count")).show(10)

+---------------------+-------+
|EngineVersion_indexed|  count|
+---------------------+-------+
|                  0.0|3845067|
|                  1.0|3675915|
|                  2.0| 265218|
|                  3.0| 212408|
|                  4.0| 160585|
|                  5.0| 136476|
|                  6.0| 120295|
|                  7.0|  93926|
|                  8.0|  70645|
|                  9.0|  68716|
+---------------------+-------+
only showing top 10 rows



In [40]:
numdf.select('EngineVersion_indexed','AppVersion_indexed','AvSigVersion_indexed').describe().show()

+-------+---------------------+------------------+--------------------+
|summary|EngineVersion_indexed|AppVersion_indexed|AvSigVersion_indexed|
+-------+---------------------+------------------+--------------------+
|  count|              8921483|           8921483|             8921483|
|   mean|   1.4386187812048736| 3.139872597414578|  418.11989946066143|
| stddev|    3.064500924641997| 6.308352188285714|   698.0236865590472|
|    min|                  0.0|               0.0|                 0.0|
|    max|                 69.0|             109.0|              8530.0|
+-------+---------------------+------------------+--------------------+



In [41]:
numdf.limit(2).toPandas().transpose()

,0,1
MachineIdentifier,055af7b21dadf6923901f06036256665,055af86f722032c3e79556be6ebe4bfd
ProductName,win8defender,win8defender
EngineVersion,1.1.15100.1,1.1.15200.1
AppVersion,4.18.1807.18075,4.18.1807.18075
AvSigVersion,1.273.1135.0,1.275.552.0
IsBeta,0,0
RtpStateBitfield,7,7
IsSxsPassiveMode,0,0
DefaultBrowsersIdentifier,None,None
AVProductStatesIdentifier,47238,53447


> All `string` (category) types are stringindexed and all `float` types are imputed. The rest of `integer` features turns out `null`-free.

#### Save it as a parquet table and done

In [42]:
import pyarrow as pa
import pyarrow.parquet as pq

In [43]:
numdf.write.option("compression","snappy").mode("overwrite")\
     .save("hdfs://master:54310/data/spark/msmalware/train_num_df.parquet.snappy")